# 비모수 검정

검정 방법 
### 단일 표본 - 1개
비모수 검정    
서열척도 : 부호검정, 부호순위검정   
명목척도 : 적합성 검정, Run 검정    
모수 검정 : 일표본 t검정      

### 대응표본(Paired) : 2개
비모수 검정   
서열척도 : 부허검정, 부호순위검정    
명목척도 : Mcnemar검정     
모수검정 : 대응표본 t검정   

### 대응표본(Paired) : k개
비모수 검정    
서열척도 : Friedman검정    
명목척도 : Cochran Q 검정    
모수검정 : paried 일원배치 분산분석(anova)  

### 독립표본(independent) : 2개
비모수 검정     
서열척도 : 순위합 검정, 만위트니 U검정   
명목척도 : 독립성검정 , 동질성 검정    
모수검정 : 독립표본 t검정    
 
### 독립표본(independet) : k개 
비모수 검정    
서열척도 : kruskal-Wallis 검정    
명목척도 : 독립성 검정, 동질성 검정    
모수검정 : 일원배치 분산분석     



## 카이제곱 검정


### 적합성 검정: 다항모집단의 비율의 차이
관측값들이 어떤 이론이나 이론적 분포를 따르고 있는지를 검정     
카이제곱분포를 이용한 검정은 기대도수가 적어도 5이상이 될 때 적용해야한다.     
  
H0 : 구해진 도수분포의 도수와 이론도수가 차이가 없다.   
H1 : 구해진 도수분포의 도수와 이론도수가 차이가 있다.   

In [2]:
# 적합성 검정 :세 후보자의 지지도가 다르다고 할수 있을까?
import numpy as np
from scipy.stats import chi2
data =np.array([60,50,40])
print(data)

[60 50 40]


In [3]:
# 가설검정 (우측검정)
# H0 : 세 후보자의 지지도는 동일하다. H1 : 세 후보자의 지지도는 차이가 있다.
m0 = data.mean()
test_a = 0.05
df = len(data) -1
chistat = sum((data - m0)**2 /m0)
sp = 1-chi2.cdf(chistat,df) # 오른쪽 검정
cv = chi2.ppf(1-test_a,df)

print("수기검정")
print(f"오른쪽 검정의 임계값 {cv}, 검정통계량 {chistat}")
print(f"유의수준 :{test_a}, 유의확률 { sp}")

수기검정
오른쪽 검정의 임계값 5.991464547107979, 검정통계량 4.0
유의수준 :0.05, 유의확률 0.1353352832366127


In [5]:
from scipy.stats import chisquare ,  chi2
stat , p = chisquare(data,m0)
print(f"검정 통계량 {stat}, p-value : {p}")
print("귀무가설을 기각할 수 없다. 세후보자의 지지도는 동일하다. ")

검정 통계량 4.0, p-value : 0.1353352832366127


### 독립성 검정 : 한 모집단 내 여러 수준의 차이

독립성 검정 혹은 교차분석이라고 한다. 다수의 인자들에 의해 분할되어 있는 데이터에서, 인자들이 관측값에 영향을 주고 있는지 여부를 검정한다.     
교차표에서 행과 열이 독립적인지 확인, 하나의 집단에서 표집한 후에 나눠봄     

H0 : 두 인자는 독립이다.(연관이 없다.)    
H1 : 두 인자는 독립이 아니다. (연관이 있다.)     

In [40]:
# 독립성 검정
# 성별 변량과 안경 착용여부 변량이 서로 독립인지 관련이 있는지 유의수준 5% 검정
import numpy as np
from pandas import DataFrame
from scipy.stats import chi2_contingency, chi2
table = DataFrame({"성별": ['남자','여자'], "안경0": [10,30], "안경x":[40,20]}).set_index('성별')
test_a = 0.05
print('데이터 확인\n',table)

데이터 확인
     안경0  안경x
성별          
남자   10   40
여자   30   20


In [35]:
# 수기검정
ttl = table.sum().sum()
exp = []
r = table.sum(axis=1).values
c = table.sum(axis=0).values
for R in r:
    for C in c:
        exp.append(R*C/ttl)
print(f'기대값\n {np.array(exp).reshape(table.shape)}')
obs = table.values.ravel()
print(f"관측값\n {np.array(obs).reshape(table.shape)}")

기대값
 [[20. 30.]
 [20. 30.]]
관측값
 [[10 40]
 [30 20]]


In [36]:
chistat = np.sum((obs - exp)** 2 / exp)
df = (table.shape[0]-1)*(table.shape[1]-1)
sp = 1 - chi2.cdf(chistat,df)
cv = chi2.ppf(1-test_a,df)
print(f"유의확률 {sp}")
print(f"오른쪽 검정의 임계값 {cv}, 검정통계량{chistat}")

유의확률 4.455709060402491e-05
오른쪽 검정의 임계값 3.841458820694124, 검정통계량16.666666666666668


In [41]:
print('라이브러리 검정')
chi2, p, df,expec = chi2_contingency(table, correction=False)
print(f"검정통계량 :{chi2}, 유의확률: {p}, 기대값 :{expec}")

라이브러리 검정
검정통계량 :16.666666666666668, 유의확률: 4.455709060405612e-05, 기대값 :[[20. 30.]
 [20. 30.]]


In [45]:
# fisher's exact test
table = DataFrame([[10,2],[3,5]], index = ['A','B'], columns = ['승','패'])
print(table)
# H0 : A와 B의 실력은 차이가 없다. H1  : A의 실력이 더 좋다.
from scipy.stats import fisher_exact
stats , p = fisher_exact(table, alternative = 'greater') # greater
print(f'검정 통계량 {stats}, 유의확률 {p}')
# print("유의 확률이 유의수준을 0.05보다 높으므로 귀무가설을 기각한다. A의 실력이 더 좋다.")
print("유의확률이 유의수준보다 높으므로 귀무가설을 기각할 수 없다. 차이가 없다")

    승  패
A  10  2
B   3  5
검정 통계량 8.333333333333334, 유의확률 0.05211558307533541
유의확률이 유의수준보다 높으므로 귀무가설을 기각할 수 없다. 차이가 없다


### 동질성검정 : 여러 (부)모집단 간 여러 수준에 대한 차이
    
서로 다른 표본 집단의 변수의 동질성(분포)를 검정하는 과정   
교차표에서 행간의 분포차이가 존재하는지 확인, 서로 다른 집단에서 따로 표집    
    
H0 : 모든 집단의 분포가 차이가 없다.(동일하다.)    
H1 : 적어도 한 집단은 분포 상 서로 차이가 있다. (동일하지 않다.)    

In [48]:
# 동질성 검정
# 프로그램 A,B,C에 대해 연령층별 시청자들의 선호가 다른지 유의수준 5%로 검정
import pandas as pd
table = pd.DataFrame({"TV": ['A','B','C'], "청년": [120,30,50], "중년":[10,75,15], "장년": [10,30,60]}).set_index('TV')
test_a = 0.05
print(table)

     청년  중년  장년
TV             
A   120  10  10
B    30  75  30
C    50  15  60


In [50]:
# H0 : 프로그램에 대한 연령 별 선호에 차이가 없다.
# H1 : 적어도 한 집단은 차이가 있다.
print("수기 검정")
ttl = table.sum().sum()
exp =[]
r = table.sum(axis=1).values
c = table.sum(axis=0).values

for R in r:
    for C in c:
        exp.append(R*C/ttl)
print(f"기대값 {np.array(exp).reshape(table.shape)}")
obs = table.values.ravel()
print(f"관찰값 {np.array(obs).reshape(table.shape)}")

수기 검정
기대값 [[70.   35.   35.  ]
 [67.5  33.75 33.75]
 [62.5  31.25 31.25]]
관찰값 [[120  10  10]
 [ 30  75  30]
 [ 50  15  60]]


In [54]:
from scipy.stats import chi2
chistat = np.sum((obs - exp)**2/exp)
df = (table.shape[0]-1) * (table.shape[1]-1)
sp = 1 - chi2.cdf(chistat,df)
cv = chi2.ppf(1-test_a,df)
print(f"오른쪽 검정의 임계값 {cv}, 검정 통계량 {chistat}")
print(f"유의수준 {test_a}, 유의확률 {sp}")

오른쪽 검정의 임계값 9.487729036781154, 검정 통계량 180.49523809523808
유의수준 0.05, 유의확률 0.0


In [60]:
print('라이브러리')
from scipy.stats import chi2_contingency
chi2, p, df, expec = chi2_contingency(table, correction = False)
print(f"검정 통계량 {chi2}, 유의확률 {p} \n기대값\n {expec}")

라이브러리
검정 통계량 180.49523809523808, 유의확률 5.836850688101578e-38 
기대값
 [[70.   35.   35.  ]
 [67.5  33.75 33.75]
 [62.5  31.25 31.25]]


### Run 검정

일표본 run검정    
   
한개의 샘플이 무작위로 추출되었는지 여부를 검정한다.     
범주형 데이터는 각 범주의 개수와 run의 개수를 사용하고,    
수치형 데이터는 중앙값을 기준으로 데이터를 이진화 한후 검정을 진행한다.   
   
H0 : 샘플이 무작위와 차이가 없다. (무작위로 추출되었다.)    
H1 : 샘플이 무작위와 차이가 잇다. (무작위로 추출되지 않았다.)     

In [62]:
# 범주형 데이터의 경우
from collections import Counter
import numpy as np
from scipy.stats import norm

#run을 세는 함수 정의
def count_run(data):
    count_run = 1
    for i, element in enumerate(data[:-1]):
        if element == data[i+1]:
            continue
        else:
            count_run += 1
    return count_run

data = ['a','a','b','b','a','a','a','a','b','b','b','b','a','b','a','a','b','b','a','a','b','b','b','a','a','b','b','a','b']
data2 = list(map(lambda x: 0 if x=='a' else 1, data))
a,b = Counter(data2)[0], Counter(data2)[1]
print(Counter(data2))

Counter({1: 15, 0: 14})


In [68]:
print('수기검정')
n1 , n2 = a,b
# n1, n2이 충분히 클 때 평균이 avg, 분산이 var인 점근적 정규분포를 따른다.
run = count_run(data2) #Run 의 개수
avg = 2*n1*n2/(n1+n2)+1
var = 2 * n1 * n2 *(2 * n1 * n2 -n1 - n2 )/ ((n1+n2)**2)/ (n1+n2-1)

# run의 총 개수에 따라서 h의 값을 정함
if run < 2*n1 * n2/(n1+n2)+0.5:
    h = 0.5
else:
    h = -0.5
#n1 , n2의 크기에 따라서 z통계량 공식을 정함
if n1 <= 20 or n2 <=20:
    Z = (run-avg + h) / np.sqrt(var) # 소표본인 경우
else:
    Z = (run-avg) / np.sqrt(var) # 대표본인 경우
sp = (1-norm.cdf(np.abs(Z)))*2
print(f"기대값 {avg}, 분산{var}, 검정통계량(Z){Z}, p-value {sp} ")

수기검정
기대값 15.482758620689655, 분산6.97384066587396, 검정통계량(Z)-0.3721438547031917, p-value 0.7097857460625616 


In [69]:
print("라이브러리 검정")
from statsmodels.sandbox.stats.runs import runstest_1samp
zstat , pval = runstest_1samp(data2)
print(f"검정 통계량 { zstat}, 유의확률{pval}")

라이브러리 검정
검정 통계량 -0.3721438547031917, 유의확률0.7097857460625617


In [70]:
# 수치형 데이터의 경우
from collections import Counter
# Run을 세는 함수 정의
def count_run(data):
    count_run = 1
    for i, element in enumerate(data[:-1]):
        if element == data[i+1]:
            continue
        else:
            count_run += 1
    return count_run
data = [50,60,70,40,30,20,10,70,80,100]
median = np.median(data)

data2 = []
for sample in data: # 값을 중앙값을 기준으로 이진화 함
    if sample >= median:
        data2.append(1)
    else:
        data2.append(0)
print(Counter(data2))

Counter({0: 5, 1: 5})


In [72]:
print("수기 검정")
n1 = Counter(data2)[0]
n2 = Counter(data2)[1]
run = count_run(data2) #Run의 총 개수
avg = 2 * n1 * n2 / (n1+n2) + 1
var = 2 * n1 *n2 * (2 * n1 * n2 -n1 -n2)/ ((n1 +n2)**2) / (n1 + n2 -1)
# 런의 총개수에 따라서 h의 값을 정함
if run < 2* n1 *n2/(n1+n2)+0.5:
    h = 0.5
else:
    h = -0.5
# n1 ,  n2의 크기에 따라서 Z통계량 공식을 정함
if n1 <= 20 or n2 <= 20 :
    Z = (run - avg + h) / np.sqrt(var)  #소표본인 경우
else:
    Z = (run - avg) / np.sqrt(var)  # 대표본인 경우
sp = (1 - norm.cdf(np.abs(Z))) * 2 
print(f'기대값 {avg}, 분산 {var}, Z검정 통계량{Z}')

수기 검정
기대값 6.0, 분산 2.2222222222222223, Z검정 통계량-1.0062305898749053


In [73]:
print("라이브러리 검정")
from statsmodels.sandbox.stats.runs import runstest_1samp
zstat ,pval = runstest_1samp(data2, cutoff = 'median')
print(f"검정 통계량 {zstat}, 유의확률 {pval}")
print("유의확률이 유의 수준보다 높으므로 귀무가설을 기각하지 못한다. 무작위로 추출되었다.")

라이브러리 검정
검정 통계량 -1.0062305898749053, 유의확률 0.31430466047385397


### 이표본 Run 검정 

H0 : 두 데이터는 같은 분포에서 왔다.    
H1 : 두 데이터는 다른 분포에서 왔다.    

In [76]:
# 수치형 데이터의 경우 
# run을 세는 함수 정의
def count_run(data):
    count_run = 1
    for i, element in enumerate(data[:-1]):
        if element == data[i+1]:
            continue
        else : 
            count_run += 1
    return count_run
data1 = [23,42,36,27,48,52,35,31]
data2 = [43,56,38,20,46,51,36]

# 라이브러리 검정을 위해 데이터 타입을 float 으로 변환시켜 주어야 한다.
data1 = list(map(lambda x : float(x), data1))
data2 = list(map(lambda x : float(x), data2))
print(Counter(data1))
print(Counter(data2))

Counter({23.0: 1, 42.0: 1, 36.0: 1, 27.0: 1, 48.0: 1, 52.0: 1, 35.0: 1, 31.0: 1})
Counter({43.0: 1, 56.0: 1, 38.0: 1, 20.0: 1, 46.0: 1, 51.0: 1, 36.0: 1})


In [77]:
data = data1 + data2 
median = np.median(data)
data3 = [1 if i >= median else 0 for i in data]  # 중앙값을 기준으로 이진화 함
print(Counter(data3))

Counter({1: 8, 0: 7})


In [83]:
# 수기검정
n1 = Counter(data3)[0]
n2 = Counter(data3)[1]
run = count_run(data3) # Run의 총 개수

avg = 2 * n1 * n2 / (n1+n2) + 1
var = 2 * n1 * n2 * (2 * n1 * n2 - n1 -n2) / ((n1 + n2)**2)/(n1 + n2 -1)

# 런의 총개수에 따라서 h의 값을 정함
if run < 2* n1 * n2/(n1 + n2)+0.5 : 
    h = 0.5
else:
    h = -0.5

# n1 , n2의 크기에 따라서 Z통계량 공식을 정함
if n1 <= 20 or n2 <= 20:
    Z = (run- avg + h) / np.sqrt(var) # 소표본인 경우
else:
    Z = (run - avg) / np.sqrt(var)
sp = (1 - norm.cdf(np.abs(Z))) * 2

print(f"기대값 {avg}, 분산 {var}, Z검정통계량 {Z}, p-value {sp}")

기대값 8.466666666666667, 분산 3.448888888888889, Z검정통계량 0.01794895396544339, p-value 0.9856795756756671


In [86]:
print("라이브러리 검정")
from statsmodels.sandbox.stats.runs import runstest_2samp
zstat, pval = runstest_2samp(data1,data2)
print(f"검정통계량 {zstat}, p-value {pval}")
# 검정 결과, p-value가 0.98로 유의수준 0.05보다 높으므로 귀무가설을 기각하지 못한다. 그러므로 같은 분포라는 가설을 기각하지 못하였다.

라이브러리 검정
ties detected
검정통계량 0.01794895396544339, p-value 0.9856795756756671


### 맥니머 검정

맥니머 검정은 이항변수인 두 변수의 대응관계(paired)가 있는 데이터 분포의 차이를 검정할 때 사용한다.     
   
H0 : 두 변수의 데이터 분포는 차이가 없다.    
H1 : 두 변수의 데이터 분포는 차이가 있다.     

In [88]:
# 프로모션 행상 전후로 상품에 대한 흥미 유무 데이터
import pandas as pd
table = pd.DataFrame([[9,12], [24,35]], index = ['전_있음','전_없음'], columns = ['후_있음','후_없음'])
print(table)

      후_있음  후_없음
전_있음     9    12
전_없음    24    35


In [89]:
# H0 : 흥미가 없다가 있게 된 경우와 있다가 없게 된 경우의 분포가 동일하다.
# H1 : 두 경우의 분포가 다르다.
print('검정') # 변화가 있는 b와 c에 주목하고 변화가 없는 대각선 값은 무시
from scipy.stats import chi2
b = table.values[0][1]
c = table.values[1][0]
stat = (b-c)**2 / (b+c)
alpha = 0.05
df = 1
pval = 1- chi2.cdf(stat,df)
print(f"검정 통계령 {stat}, p-value {pval}")

검정
검정 통계령 4.0, p-value 0.04550026389635853


In [90]:
from statsmodels.stats.contingency_tables import mcnemar
# 파라미터 exact 값이 True이면 이항분포, false면 카이제곱분포를 사용한다.

print('라이브러리')
mc = mcnemar(table.values, exact=False, correction = False)
print(f'검정 통계량 {mc.statistic}, 유의확률 {mc.pvalue}')
# 유의확률이 0.045로 유의수준 0.05보다 낮아 귀무가설을 기각합니다. 이벤트의 효과가 있었다. 

라이브러리
검정 통계량 4.0, 유의확률 0.04550026389635857


### 코크란 Q검정

코크란 Q검정은 이항 변수인 세변수 이상의 대응관계가 있는 데이터으 분포의 차이를 검정할 댸 사용(paired one-way anova의 비모수 검정)
      
H0 : 각 변수의 데이터 분포는 차이가 없다.         
H1 : 적어도 한 쌍의 변수의 데이터 분포는 차이가 있다.       

In [92]:
# 연예인 3명에 대한 호감도 데이터를 얻기 위해 8명에게 설문조사를 실시했다. 연예인에 대한 호감 비율에 차이가 있는가?
import pandas as pd
import numpy as np
from scipy.stats import chi2
table = DataFrame([[0,1,0,1,0,0,0,0],[1,1,0,1,0,0,1,1],[0,1,1,1,1,1,1,1]], index=['가수1','가수2','가수3'], columns=[1,2,3,4,5,6,7,8]).T 
print(table)

   가수1  가수2  가수3
1    0    1    0
2    1    1    1
3    0    0    1
4    1    1    1
5    0    0    1
6    0    0    1
7    0    1    1
8    0    1    1


In [95]:
# H0 : 모든 연예인에 대한 호감도 비율이 차이가 없다(모든 연예인의 호감도가 같다)
# H1 : 적어도 한쌍의 연예인은 호감도 비율이 차이가 있다. (적어도 한 쌍 다르다)
print("수기 검정")
n = table.shape[0] # 표본 개수
k = table.shape[1] # 범주 개수
df = k - 1
k_sums = np.array(table.sum())
n_sums = np.array(table.sum(axis=1))
Q1 = k * sum(k_sums**2) - sum(k_sums)**2
Q2 = k * sum(n_sums)  - sum(n_sums**2)
stat = df * Q1 /Q2
pval = 1-chi2.cdf(stat,df)
print(f"검정 통계량 :{stat}, 유의확률 {pval}")

수기 검정
검정 통계량 :6.333333333333333, 유의확률 0.04214384350927636


In [97]:
# 라이브러리
from statsmodels.stats.contingency_tables import cochrans_q
ccq = cochrans_q(table)
print(f"검정통계량 : {ccq.statistic}, 유의확률 :{ccq.pvalue}")
print("유의확률이 유의수준보다 낮아 귀무가설을 기간한다. 적어도 한쌍의 연예인은 호감도비율이 차이가 있다.")

검정통계량 : 6.333333333333333, 유의확률 :0.042143843509276406


In [104]:
# 연예인 모드 쌍의 맥니머 검정을 통해 가수 1- 가수 3의 호감도 차이가 있음을 확인
from itertools import combinations
col_comp = list(combinations(table.columns,2))

import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar
for s1,s2 in col_comp:
    ct = pd.crosstab(table.loc[:,s1], table.loc[:,s2])
    mc = mcnemar(ct.values, exact=False, correction = False)
    stat, p = mc.statistic, mc.pvalue
    msg = f'{s1}{s2} : 검정통계량 {stat}, p-value {p}'    
    if p < 0.05:
        print(msg + '   ***')
    else:
        print(msg)

가수1가수2 : 검정통계량 3.0, p-value 0.08326451666355042
가수1가수3 : 검정통계량 5.0, p-value 0.025347318677468325   ***
가수2가수3 : 검정통계량 1.0, p-value 0.31731050786291115


### 일표본 부호검정 : 이항분포, z분포

표본 수가 100보다 낮으면 부호검정 통꼐량은 이항분포를 다르고 100보다 크면 정규화된 정규분포를 따른다.   

일표본 t검정에 대응한다.

In [110]:
# 데이터의 중앙값으로 알려진 M0 = 200 일 때 가설검정
# H0 : 데이터의 중앙값은 200과 차이가 없다. 
# H1 : 데이터의 중앙값은 200과 차이가 있다.

print("수기 검정")
import numpy as np
data = np.array([203,204,197,195,201,205,198,199,194,207]) # 데이터
M0 = 200 # 귀무가설의 중앙값

# 검정통꼐량 계산하기
df = pd.DataFrame(data-M0, columns = ['d'])
df = df.query("d !=0")
plus = len(df.query("d > 0")) # 통계량 
minus = len(df.query("d < 0 "))
n = len(df) # 유효 데이터 개수
print(f"유효한 데이터 개수 : {n}, 검정통계량 {plus}")

수기 검정
유효한 데이터 개수 : 10, 검정통계량 5


In [114]:
# plus  부호의 개수, 즉 검정통계량 B는 B(n, p=0.5)인 이항분포를 따른다.
from scipy.stats import binom

p = 0.5
result = DataFrame()
for i in range(0, n+1):
    result.loc[i,'X'] = i
    result.loc[i,'prob'] = binom.pmf(i,n,p)
result['value'] = result['X'] * result['prob']
print("이항분포 기반\n ", result)
    

이항분포 기반
         X      prob     value
0    0.0  0.000977  0.000000
1    1.0  0.009766  0.009766
2    2.0  0.043945  0.087891
3    3.0  0.117188  0.351562
4    4.0  0.205078  0.820312
5    5.0  0.246094  1.230469
6    6.0  0.205078  1.230469
7    7.0  0.117187  0.820312
8    8.0  0.043945  0.351563
9    9.0  0.009766  0.087891
10  10.0  0.000977  0.009766


In [115]:
# 기대값과 분산
mean = result.sum()['value']
var = sum((result['X']-mean)**2 * result['prob'])
print(f"기대값 {mean}, 분산 {var}, (={n*p},{n*p*(1-p)}) ")

기대값 4.999999999999999, 분산 2.5, (=5.0,2.5) 


In [116]:
# 검정 통계량과 임계치
test_a = 0.05 #유의수준 5% 양측 검정
start = int(binom.ppf(test_a/2,n,p))
end = int(binom.ppf(1-test_a/2,n,p))
print(f"검정통계량 {plus}, 임계치 {start}~ {end}")

검정통계량 5, 임계치 2~ 8


In [120]:
# 정규분포 근사
from scipy.stats import norm
mean2 = n/2
var2 = n/4
s = np.sqrt(var2)
zstat = (plus - mean2)/s # plus를 대입
ways = 'two'
if ways == 'two':
    sp = (1-norm.cdf(np.abs(zstat)))*2 
    cv = norm.ppf(1-test_a/2)
    cv = f'+/- {cv}'
elif ways== 'one-right':
    sp = 1-norm.cdf(zstat)
    cv = norm.ppf(1-test_a)
    cv = f'{cv}'
elif ways == 'one-left':
    sp = norm.cdf(zstat)
    cv = norm.ppf(test_a)
    cv = f'{cv}'

print(f"정규분포기반 기대값 : {mean2}, 분산 : {var2}")
print(f"검정통계량 :{zstat}, 임계치 : {cv}")
print(f"유의수준: {test_a}, 유의확률 {sp}")
print("검정 결과, 귀무가설을 채택하여 평균의 차이는 200과 차이가 없다.")

정규분포기반 기대값 : 5.0, 분산 : 2.5
검정통계량 :0.0, 임계치 : +/- 1.959963984540054
유의수준: 0.05, 유의확률 1.0
검정 결과, 귀무가설을 채택하여 평균의 차이는 200과 차이가 없다.


### 이표본 부호 검정 : 이항분포, Z분포

n<100 이면 이항분포를 따르고 n>100 이면 정규분포를 따른다

대응표본 t검정에 대응한다.

In [122]:
# A, B의 5점 척도 만족도 설문조사로부터 A의 만족도가 더 높다고 할 수 있는지 검정
# H0 : A와 B의 만족도는 차이가 없다.
# H1 : A의 만족도가 더 높다.

print("검정")
data1 = np.array([4,3,5,2,1,3,4,3]) #A
data2 = np.array([3,2,3,1,2,2,2,2]) #B
M0 = 0 
df = pd.DataFrame(data1 - data2 ,columns = ['d'])
df = df.query("d != 0")
plus = len(df.query("d > 0"))
minus = len(df.query("d < 0"))
n = len(df)
print(f" 유효한 데이터 개수 {n},plus 개수 (=검정통계량) {plus}")

검정
 유효한 데이터 개수 8,plus 개수 (=검정통계량) 7


In [123]:
# plus 부호의 개수 X는 (n, p = 0.5)인 이항분포를 따른다.
from scipy.stats import binom
p = 0.5
result = DataFrame()
for i in range(0,n+1):
    result.loc[i,'X'] = i
    result.loc[i, 'prob'] = binom.pmf(i,n,p)
result['value'] = result['X'] * result['prob']
print(result)

     X      prob    value
0  0.0  0.003906  0.00000
1  1.0  0.031250  0.03125
2  2.0  0.109375  0.21875
3  3.0  0.218750  0.65625
4  4.0  0.273437  1.09375
5  5.0  0.218750  1.09375
6  6.0  0.109375  0.65625
7  7.0  0.031250  0.21875
8  8.0  0.003906  0.03125


In [124]:
# 기대값과 분산
mean = result.sum()['value']
var = sum((result['X'] - mean) **2 * result['prob'])
print(f"기대값 {mean}, 분산 {var}, (={n*p}~ {n*p*(1-p)})")

기대값 3.999999999999999, 분산 2.0, (=4.0~ 2.0)


In [126]:
# 검정 통계량과 임게치
test_a = 0.05 #유의수준 5%, 양측검정
start = int(binom.ppf(test_a/2,n,p))
end = int(binom.ppf(1-test_a/2,n,p))
print(f"검정통계량 {plus}, 임계치 {start},{end}")

검정통계량 7, 임계치 1,7


In [128]:
# 정규분포 근사
from scipy.stats import norm
mean2 =n/2
var2 = n/4
s = np.sqrt(var2)
zstat = (plus - mean2)/s 
ways = 'one-right'
if ways =='two':
    sp = (1-norm.cdf(np.abs(zstat)))* 2
    cv = norm.ppf(1-test_a/2)
    cv = f'+/- {cv}'
elif ways == 'one-right':
    sp = 1-norm.cdf(zstat)
    cv = norm.ppf(1-test_a)
    cv = f'{cv}'
elif ways == 'one-left':
    sp = norm.cdf(zstat)
    cv = norm.ppf(test_a)
    cv = f'{cv}'

print(f"기대값 {mean2}, 분산 {var2}")
print(f"검정통계량 {zstat}, 임계치 {cv}")
print(f"유의수준 {test_a}, 유의확률{sp}")
print("검정결과, 유의확률이 유의수준보다 낮으므로 귀무가설을 기각한다. A의 만족도가 더 높다.")

기대값 4.0, 분산 2.0
검정통계량 2.1213203435596424, 임계치 1.6448536269514722
유의수준 0.05, 유의확률0.01694742676234462
검정결과, 유의확률이 유의수준보다 낮으므로 귀무가설을 기각한다. A의 만족도가 더 높다.


### 일표본 윌콕슨 부호 순위 검정 :  윌콕슨 부호순위 검정표, Z분포

n<20 이면 윌콕슨 순위합 분포를 따르고 n > 20이면 정규분포를 근사한다.

일표본 t 검정에 대응한다.

In [130]:
# 데이터의 중앙값으로 알려진 M0 = 200 일때 가설검정
# H0 : 데이터의 중앙값은 200과 차이가 없다.
# H1 :  데이터의 중앙값은 200과 차이가 있다.

print("수기 검정")
data = np.array([203,204,197,195,201,205,198,199,194,207]) # 데이터
M0 = 200 # 중앙값
d = data - M0
table = pd.DataFrame(d,columns = ['d'])
n = len(table)
table['sign'] = np.sign(table['d'])
table['abs_d'] = table['d'].abs()
table['rank'] = table['abs_d'].rank(method = 'average')
print(table)

수기 검정
   d  sign  abs_d  rank
0  3     1      3   4.5
1  4     1      4   6.0
2 -3    -1      3   4.5
3 -5    -1      5   7.5
4  1     1      1   1.5
5  5     1      5   7.5
6 -2    -1      2   3.0
7 -1    -1      1   1.5
8 -6    -1      6   9.0
9  7     1      7  10.0


In [132]:
plus = table.query("sign==1")['rank'].sum()
minus = table.query("sign==-1")['rank'].sum()
stat = np.minimum(plus, minus) # 부호의 개수가 더 적은 것을 검정통계량으로 사용
print("윌콕슨 부호 순위 분포 기반")
print(f"유효한 데이터 개수 {n}, 검정 통계량 {stat}")


윌콕슨 부호 순위 분포 기반
유효한 데이터 개수 10, 검정 통계량 25.5


In [135]:
print('라이브러리 검정')
from scipy.stats import wilcoxon
stat, p = wilcoxon([M0 for i in range(len(data))], data)
print(f"검정통계량 {stat}, p-value {p} ")

라이브러리 검정
검정통계량 25.5, p-value 0.845703125 


In [141]:
# 정규 근사
wstat = np.minimum(plus, minus) #작은쪽을 검정통계량 
mean = n*(n+1)/4
var = n * (n+1)*(2*n+1)/24
s = np.sqrt(var)
zstat = (wstat - mean) /s 
ways = 'two'
if ways == 'two':
    sp = (1-norm.cdf(np.abs(zstat)))*2 
    cv = norm.ppf(1-test_a/2)
    cv = f'+/-{cv}'
elif ways =='one-right':
    sp = 1-norm.cdf(zstat)
    cv = norm.ppf(1-test_a)
    cv = f'{cv}'
elif ways =='one-left':
    sp = norm.cdf(zstat)
    cv = norm.ppf(test_a)
    cv = f'{cv}'

print(f"정규분포 근사 기대값 {mean}, 분산은 {var}")
print(f"검정통계량 {zstat}, 임계치 {cv}")
print(f"유의수준 :{test_a}, 유의확률 {sp}")

# 유의확률이 유의수준보다 높아 귀무가설을 기각하지 못하였다 그러므로 데이터의 중앙값은 200과 차이가 없다.

정규분포 근사 기대값 27.5, 분산은 96.25
검정통계량 -0.20385887657505022, 임계치 +/-1.959963984540054
유의수준 :0.05, 유의확률 0.8384637819224636


### 이표본 윌콕슨 부호순위 검정 : 윌콕슨 부호순위 검정표, Z분포

n < 20이면 윌콕슨 순위합 분포를 따르고, n > 20이면 정규분포를 근사한다.

대응표본 t검정에 대응한다.

In [142]:
# 동일한 피험자 8명에게 맥박을 2번 측정하였을 때, 
# H0 : 1번째 측정값과 2번째 측정값은 차이가 없다. H1 : 차이가 있다.
print("수기 검정")
data1 = np.array([79,96,85,69,88,75,83,88])
data2 = np.array([70,88,73,74,75,79,77,81])
d = data1 -data2
table = pd.DataFrame(d, columns = ['d'])
table = table.query("d != 0")
n = len(table)
table['sign'] = np.sign(table['d'])
table['abs_d'] = table['d'].abs()
table['rank'] = table['abs_d'].rank(method='average')
print(f"부호순위 계산표 : \n{table}")

수기 검정
부호순위 계산표 : 
    d  sign  abs_d  rank
0   9     1      9   6.0
1   8     1      8   5.0
2  12     1     12   7.0
3  -5    -1      5   2.0
4  13     1     13   8.0
5  -4    -1      4   1.0
6   6     1      6   3.0
7   7     1      7   4.0


In [144]:
plus = table.query("sign == 1")['rank'].sum()
minus = table.query("sign== -1")['rank'].sum()
stat = np.minimum(plus,minus) # 부호의 개수가 더 적은 것을 검정통계량으로 사용
print("윌콕슨 부호 순위 분포 기반" )
print(f"유효한 데이터 개수 {n}, 검정통계량 {stat}")

윌콕슨 부호 순위 분포 기반
유효한 데이터 개수 8, 검정통계량 3.0


In [145]:
print("라이브러리 검정")
from scipy.stats import wilcoxon
stat , p = wilcoxon(data1,data2,zero_method='wilcox')
print(f"검정 통계량{stat}, p-value {p}")

라이브러리 검정
검정 통계량3.0, p-value 0.0390625


In [147]:
# 정규 근사 
wstat = np.minimum(plus, minus) # 작은쪽을 검정 통계량으로
mean = n * (n+1) / 4
var = n * (n+1) * (2 * n +1) / 24
s = np.sqrt(var)
zstat = (wstat - mean)/s
ways = 'two'
if ways =='two':
    sp = (1-norm.cdf(np.abs(zstat)))* 2
    cv = norm.ppf(1-test_a/2)
    cv = f' +/- {cv}'
elif ways =='one-right':
    sp = 1-norm.cdf(zstat)
    cv = norm.ppf(1-test_a)
    cv = f'{cv}'
elif ways == 'one-left':
    sp = norm.cdf(zstat)
    cv = norm.ppf(test_a)
    cv = f'{cv}'

print(f"정규분포 기반 기대값 {mean}, 분산 {var}")
print(f"검정통계량 {zstat}, 임계치 {cv}")
print(f"유의 수준 {test_a}, 유의 확률 {sp}")
print("검정결과 귀무가설을 기각하여 1번째 측정값과 2번째 측정값의 차이가 있다.")

정규분포 기반 기대값 18.0, 분산 51.0
검정통계량 -2.100420126042015, 임계치  +/- 1.959963984540054
유의 수준 0.05, 유의 확률 0.03569190011680434
검정결과 귀무가설을 기각하여 1번째 측정값과 2번째 측정값의 차이가 있다.


### 윌콕슨 순위합 검정(만 위트니U 검정) :  윌콕슨 순위함 검정표, Z분포

n <25 이면 순위합 분포를 따르고, n > 25이면 정규분포에 근사한다.

독립표본 t 검정에 대응한다.     
만위트니U검정와 동일한 결과를 얻는다.


In [149]:
# 팀 별 영업성적이 차이가 있는지 검정
# H0 : 성적 차이가 없다. 두 A, B 중앙값의 차이는 0이다.
# H1 : 성적차이가 있다. 두 A,B 중앙값의 차이는 0이 아니다.

data1 = [87,75,65,95,90,81,93]
data2 = [57,85,90,83,87,71]

n1 = len(data1)
n2 = len(data2)
M1 = np.median(data1)
M2 = np.median(data2)

print("수기 검정")
table1 = pd.DataFrame(data1, columns = ['data'])
table1['M'] = M1
table2 = pd.DataFrame(data2, columns = ['data'])
table2['M'] = M2
table = pd.concat([table1,table2])
table['rank']= table['data'].rank(method='average')
print(table)


수기 검정
   data     M  rank
0    87  87.0   8.5
1    75  87.0   4.0
2    65  87.0   2.0
3    95  87.0  13.0
4    90  87.0  10.5
5    81  87.0   5.0
6    93  87.0  12.0
0    57  84.0   1.0
1    85  84.0   7.0
2    90  84.0  10.5
3    83  84.0   6.0
4    87  84.0   8.5
5    71  84.0   3.0


In [151]:
W1 = table.query("M==@M1")['rank'].sum() # 순위합 검정통계량1
W2 = table.query("M==@M2")['rank'].sum()
U1 = W1 - n1 * (n1+1)/2 # 만위트니 검정 통계량
U2 = W2 - n2 * (n2+1)/2 
print(f"자료의 개수 { n1,n2}")
print(f"순위합 검정 통계량 {W1, W2}")
print(f"만위트니 검정 통계량 {U1,U2}")

자료의 개수 (7, 6)
순위합 검정 통계량 (55.0, 36.0)
만위트니 검정 통계량 (27.0, 15.0)


In [153]:
print("라이브러리 검정")
from scipy.stats import ranksums, mannwhitneyu
zstat, p = ranksums(data1,data2) # default : alternative = 'two-sided'
print(f"순위합 정규근사 검정 통계량{zstat}, p-value {p}")
mstat, p = mannwhitneyu(data1,data2, alternative='two-sided')
print(f"만 위트니 검정통계량 {mstat}, p-value {p}")

라이브러리 검정
순위합 정규근사 검정 통계량0.8571428571428571, p-value 0.39136593830755195
만 위트니 검정통계량 27.0, p-value 0.43076586096421876


In [162]:
# 정규 근사 
mean = n1 * n2 /2
var =n1 * n2 * (n1+n2+1)/12
s = np.sqrt(var)
Z1 = (U1 -mean)/s # 만위트니 검정 통게량 
Z2 = (U2 - mean)/s 
test_a = 0.05
zstat = Z1 # Z1과 Z2는 부호만 다르고 같은 값
ways= 'two'
if ways =='two':
    sp = (1-norm.cdf(np.abs(zstat)))*2
    cv = norm.ppf(1-test_a/2)
    cv = f'+/- {cv}'
elif ways =='one-right':
    sp = 1-norm.cdf(zstat)
    cv = norm.ppf(1-test_a)
    cv = f'{cv}'
elif ways == 'one-left':
    sp = norm.cdf(zstat)
    cv = norm.ppf(test_a)
    cv = f'{cv}'

print(f"정규 분포 기반 가설 검정 기대값 : {mean}, 분산 : {var}")
print(f"검정통계량 {zstat}, 임계값{cv}")
print(f"유의수준 {test_a}, 유의확률 {sp}")

print("검정 결과, 귀무가설을 기각하지 못한다. 그러므로 두 성적의 차이는 없다고 결론을 얻었다.")

정규 분포 기반 가설 검정 기대값 : 21.0, 분산 : 49.0
검정통계량 0.8571428571428571, 임계값+/- 1.959963984540054
유의수준 0.05, 유의확률 0.39136593830755206
검정 결과, 귀무가설을 기각하지 못한다. 그러므로 두 성적의 차이는 없다고 결론을 얻었다.


### K 표본 순위 데이터 검정
### 크러스컬 윌리스 검정 :크러스컬 윌리스 검정표, 카이제곱 분포
세 변수 이상의 대응 관계가 없는 데이터의 차이를 검정한다. 즉 대응 관계가 없는 일원배치분산분석의 비모수 버전    
독립표본 one-way anova이다.     
사후검정은 윌콕슨 순위합 검정으로 진행할 수 있다.    

In [163]:
# A,B,C 세 사람의 모의고사 성적을 통해 성취도의 차이가 있는지 검정
# H0 : 세 사람의 성취도는 차이가 없다.
# H1 : 적어도 한쌍의 성취도는 차이가 있다.
table = pd.DataFrame([[69,67,65,59,66],[56,63,55,40],[71,72,70,75]],index = ['A','B','C'])
print(table)

    0   1   2   3     4
A  69  67  65  59  66.0
B  56  63  55  40   NaN
C  71  72  70  75   NaN


In [172]:
print("수기 검정")
ranktable = pd.DataFrame(table.values.ravel()).rank(ascending=False).values.reshape(table.shape)
print(ranktable)

수기 검정
[[ 5.  6.  8. 10.  7.]
 [11.  9. 12. 13. nan]
 [ 3.  2.  4.  1. nan]]


In [175]:
N = [] #범저별 데이터 개수
R = [] #범주별 순위합
for i in range(len(table)):
    row = table.iloc[i,:] # 데이터프레임일대 행 추출
    rank = ranktable[i,:] # 배열일 때 행 추출
    ni = 0 
    ri = 0
    for d, r in zip(row, rank):
        if np.isnan(d) == False:
            ni += 1
            ri += r # 범주별 순위합 구하기
    N.append(ni)
    R.append(ri)

#리스트를 array로 변환
N = np.array(N)
R = np.array(R)
df = len(N)-1 

#가설 검정 (우측 검정)
n = sum(N)
H = 12/(n * (n+1)) * sum(R**2/N) - 3*(n+1) # 검정통계량
pval = 1-chi2.cdf(H,df)
print(f"검정통계량 {H}, p-value {pval}")

검정통계량 10.117582417582419, p-value 0.006353234607321956


In [179]:
from scipy.stats import kruskal
print("라이브러리 검정")
# nan_policy = 'omit' 으로 설정하면 nan 값은 제거하고 자동 계산
stat, p = kruskal(table.values[0], table.values[1], table.values[2], nan_policy = 'omit')
print(f"검정통계량 {stat}, p-value {p}")

print("검정결과 귀무가설을 기각하여 적어도 한쌍의 성취도 차이가 존재한다는 결론을 얻었다.")

라이브러리 검정
검정통계량 10.117582417582419, p-value 0.006353234607321947
검정결과 귀무가설을 기각하여 적어도 한쌍의 성취도 차이가 존재한다는 결론을 얻었다.


In [190]:
table=table.drop(columns=4)

,0,1,2,3
A,69,67,65,59
B,56,63,55,40
C,71,72,70,75


In [194]:
# 사후검정 : 모든 조합 별 윌콕슨 순위합 검정에서는 A -C의 조합에서 유의한 차이가 있는것으로 나타났다.
# table=table.drop(columns=4)
from itertools import combinations
col_comp = list(combinations(table.index,2))

from scipy.stats import ranksums
for s1, s2 in col_comp:
    stat, p = ranksums(table.loc[s1,:], table.loc[s2,:])
    msg = f" {s1} ~ {s2} : 검정통계량{stat}, p-value {p}"
    if p < 0.05:
        print(msg + "/*****")
    else: 
        print(msg)
# 귀무가설을 기각한다. 차이가 있다. 

 A ~ B : 검정통계량2.0207259421636903, p-value 0.043308142810791955/*****
 A ~ C : 검정통계량-2.3094010767585034, p-value 0.020921335337794014/*****
 B ~ C : 검정통계량-2.3094010767585034, p-value 0.020921335337794014/*****


### 프리드먼 검정 : 프리드먼 검정표 , 카이제곱분포
세 변수 이상의 대응관계가 있는 데이터의 차이를 검정한다. 즉 paired one-way anova     
사후검정은 윌콕슨의 부호순위검정으로 진행



In [197]:
# 운전자 A,B,C,D의 운전 점수에서 차이가 있는지 검정
# H0 : 네 운전자의 운전 점수는 차이가 없다.
# H1 : 적어도 한쌍의 운전자의 운전 점수는 차이가 있다.
from scipy.stats import friedmanchisquare
table = pd.DataFrame([[4,2,5],[3,5,2],[5,4,4],[1,1,3]], index=['A','B','C','D'])
print(table)

   0  1  2
A  4  2  5
B  3  5  2
C  5  4  4
D  1  1  3


In [198]:
# 수기검정
srtable = table.rank(ascending=False)

In [199]:
srtable

,0,1,2
A,2.0,3.0,1.0
B,3.0,1.0,4.0
C,1.0,2.0,2.0
D,4.0,4.0,3.0


In [205]:
R = srtable.sum(axis=1).values # 행범주별 순위합
n = table.shape[1]
k = table.shape[0]
df = k- 1

# 가설검정(우측검정)
Q = 12/(n*k*(k+1) ) * sum(R**2) -3 * n * (k+1) # 검정통계량
pval = 1-chi2.cdf(Q,df) # 우측
print(f"검정통계량 {Q}, p-value {pval}")

검정통계량 4.200000000000003, p-value 0.24066188520961496


In [209]:
# 라이브러리
stat, p =friedmanchisquare(table.values[0], table.values[1],table.values[2], table.values[3])
print(f"검정통계량{stat}, p-value {p}")

검정통계량4.200000000000003, p-value 0.24066188520961498


In [211]:
# 사후검정 : 모든 조합 별 윌콕슨 부호순위 검정에서도 모든 쌍에서 유의한 차이가 없는 것으로 나타났다.

from itertools import combinations
col_comp = list(combinations(table.index,2))

from scipy.stats import wilcoxon
for s1,s2 in col_comp:
    stat,p = wilcoxon(table.loc[s1,:], table.loc[s2,:], zero_method='wilcox')
    msg = f"{s1}-{s2} 검정통계량 {stat}, p-value {p}"
    if p< 0.05:
        print(msg + "****")
    else:
        print(msg)

## 모두 귀무가설을 기각하지 않으므로 차이가 없다. 

A-B 검정통계량 2.5, p-value 1.0
A-C 검정통계량 1.5, p-value 0.5
A-D 검정통계량 0.0, p-value 0.25
B-C 검정통계량 1.0, p-value 0.5
B-D 검정통계량 1.0, p-value 0.5
C-D 검정통계량 0.0, p-value 0.25
